<a href="https://colab.research.google.com/github/stefanoridolfi/ML_From_scratch/blob/master/ch_1_12_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# FROM ch_1_1.ipynb##########################
##############################################
import pandas as pd
from random import seed
from random import randrange
from csv import reader
from math import sqrt,exp, pi

''' if no headers in csv
def load_csv(filename_url):
  CSV_url=filename_url
  pdfile=pd.read_csv(CSV_url,header=None)
  dataset=pdfile.values.tolist()
  return dataset
  '''
'''if header in csv
#CVS with headers ##################
def load_csv(filename_url):
  CSV_url=filename_url
  pdfile=pd.read_csv(CSV_url,sep=';',nrows=48)
  #pdfile=pd.read_csv(CSV_url,sep=';')
  headers=pdfile.head()
  dataset=pdfile.values.tolist()
  return dataset, list(headers)
  '''
def load_csv(filename_url):
  CSV_url=filename_url
  #pdfile=pd.read_csv(CSV_url,header=None,skiprows=749, nrows=30)
  pdfile=pd.read_csv(CSV_url,header=None)
  dataset=pdfile.values.tolist()
  return dataset

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def train_test_split(dataset, split):
  train  = list()
  test=list()
  train_size = split * len(dataset)
  dataset_copy = list(dataset)
  while len(train) < train_size:
    index = randrange(len(dataset_copy))
    elem=dataset_copy[index]
    dataset_copy.pop(index)
    train.append(elem)
    test=dataset_copy
  return train, test

# Split the dataset by class values, returns a dictionary
def separate_by_class(dataset):
  separated=dict()
  for row in dataset:
    classe=row[-1]
    if classe not in separated:
      separated[classe]=list()
    separated[classe].append(row)
  return separated  

from math import sqrt
def mean(numeri):
  return sum(numeri)/float(len(numeri))

def stdev(numeri):
  avg=mean(numeri)
  varianza=sum([(num-avg)**2 for num in numeri])/float(len(numeri)-1)
  return sqrt(varianza)

def summarize_dataset(dataset):
  zippato=list(zip(*dataset))
  summaries=[]
  for elem in zippato:
    print("elem=",elem)
    if is_number(elem[0]):
      summaries.append([mean(elem),stdev(elem),len(elem)])
  #del summaries[-1]
  return summaries


def summarize_by_class(dataset):
  summaries=dict()
  separated=separate_by_class(dataset)
  for chiave in separated:
    summaries[chiave]=summarize_dataset(separated[chiave])
  return summaries

#Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
  exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
  return (1 / (sqrt(2 * pi) * stdev)) * exponent
##################

# Dataset are in https://github.com/jbrownlee/Datasets

#dataset Banknote
CSV_url='https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'

dataset= load_csv(CSV_url)
# dataset for functions test
''' 
dataset = [[3.393533211,2.331273381,0],
[3.110073483,1.781539638,0],
[1.343808831,3.368360954,0],
[3.582294042,4.67917911,0],
[2.280362439,2.866990263,0],
[7.423436942,4.696522875,1],
[5.745051997,3.533989803,1],
[9.172168622,2.511101045,1],
[7.792783481,3.424088941,1],
[7.939820817,0.791637231,1]]
'''
print('Loaded data file {0} with {1} rows and {2} columns\n\n'.format(CSV_url, len(dataset),len(dataset[0])))
#print("classi",separate_by_class(dataset))

separated=separate_by_class(dataset)
print("separated by class:",separated)
#for chiave in separated:
 # print("chiave in separated",chiave)
  #print("valori di",chiave," in separated",separated[chiave])
  #for row in separated[chiave]:
   # print("row di",chiave,"= ",row)

summaries=summarize_by_class(dataset)
print("summaries by class:", summaries)
#for label in summaries:
  #print(label)
 # for row in summaries[label]:
  #  print(row)
# summaries are the mean , stdev of each variable per each class, it will be used to calculated mean and stdev using train dataset
# for each class there are mean stdev and lenght of each variable: [2.7420144012, 0.9265683289298018, 5] is for X1 in class 0, [3.0054686692, 1.1073295894898725, 5] is for X2 ib class 1
#summaries[label][0][2] is the number of items in class summaries[label], in this case is 5 for class 0 and 5 for class 0
# summaries.items are the couple of label:value in dict summaries
# total_row is teh total number of item, for all classes. It is calculated as sum of item in each class 
total_rows=sum([summaries[key][0][2] for key in summaries])
print("total_rows=",total_rows)
# probabilty of each class: number of item in each class/float(total_row). it is a dictionary
probabilities_class=dict()
for key in summaries:
  probabilities_class[key] = summaries[key][0][2]/float(total_rows)
print("probabilities_class=",probabilities_class)
# calculated the probability that a specif row in test dataset belongs to a class. we molt the prob of the class, probabilities_class to probablity of teh single 
# variable
def calculate_class_probabilities(summaries, row):
  probabilities= dict()
  for key, class_summaries in summaries.items():
    probabilities[key]=probabilities_class[key]
    for i in range(len(class_summaries)):
      mean, stdev, _ = class_summaries[i]
      probabilities[key] *= calculate_probability(row[i], mean, stdev)
  return probabilities
prob=calculate_class_probabilities(summaries,dataset[0])
print("prob=", prob)

#Predict the class for a given row
def predict(summaries, row):
  probabilities = calculate_class_probabilities(summaries, row)
  best_label, best_prob = None, -1
  for class_value, probability in probabilities.items():
    if probability > best_prob:
      best_prob = probability
      best_label = class_value
  return best_label

#Naive Bayes Algorithm
def naive_bayes(train, test):
  summarize = summarize_by_class(train)
  predictions = list()
  for row in test:
    output = predict(summarize, row)
    predictions.append(output)
  return(predictions)

# Calculate accuracy percentage
def accuracy_metric(test, predicted):
  actual = [row[-1] for row in test ]
  correct = 0
  for i in range(len(actual)):
    if actual[i] == predicted[i]:
      correct += 1
  return correct / float(len(actual)) * 100.0

split=0.6
train, test = train_test_split(dataset, split)
predicted=naive_bayes(train, test)
print("predicted_all",predicted)
acc=accuracy_metric(test, predicted)
print("Accuracy=",acc)

Loaded data file https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv with 150 rows and 5 columns


separated by class: {'Iris-setosa': [[5.1, 3.5, 1.4, 0.2, 'Iris-setosa'], [4.9, 3.0, 1.4, 0.2, 'Iris-setosa'], [4.7, 3.2, 1.3, 0.2, 'Iris-setosa'], [4.6, 3.1, 1.5, 0.2, 'Iris-setosa'], [5.0, 3.6, 1.4, 0.2, 'Iris-setosa'], [5.4, 3.9, 1.7, 0.4, 'Iris-setosa'], [4.6, 3.4, 1.4, 0.3, 'Iris-setosa'], [5.0, 3.4, 1.5, 0.2, 'Iris-setosa'], [4.4, 2.9, 1.4, 0.2, 'Iris-setosa'], [4.9, 3.1, 1.5, 0.1, 'Iris-setosa'], [5.4, 3.7, 1.5, 0.2, 'Iris-setosa'], [4.8, 3.4, 1.6, 0.2, 'Iris-setosa'], [4.8, 3.0, 1.4, 0.1, 'Iris-setosa'], [4.3, 3.0, 1.1, 0.1, 'Iris-setosa'], [5.8, 4.0, 1.2, 0.2, 'Iris-setosa'], [5.7, 4.4, 1.5, 0.4, 'Iris-setosa'], [5.4, 3.9, 1.3, 0.4, 'Iris-setosa'], [5.1, 3.5, 1.4, 0.3, 'Iris-setosa'], [5.7, 3.8, 1.7, 0.3, 'Iris-setosa'], [5.1, 3.8, 1.5, 0.3, 'Iris-setosa'], [5.4, 3.4, 1.7, 0.2, 'Iris-setosa'], [5.1, 3.7, 1.5, 0.4, 'Iris-setosa'], [4.6, 3.6, 1.0, 0.2, 'Iris-setosa